In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import os

def gaussian_pdf(x, mu, sigma):
    """Calculate Gaussian probability density function"""
    return np.exp(-0.5 * ((x - mu) / sigma)**2) / (sigma * np.sqrt(2 * np.pi))

def gaussian_score(x, mu, sigma):
    """Calculate score function (gradient of log pdf) for Gaussian: ∇log p(x) = -(x-μ)/σ²"""
    return -(x - mu) / (sigma**2)

def mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1=0.5):
    """Calculate mixture of two Gaussians"""
    return w1 * gaussian_pdf(x, mu1, sigma1) + (1-w1) * gaussian_pdf(x, mu2, sigma2)

def mixture_score(x, mu1, sigma1, mu2, sigma2, w1=0.5):
    """Calculate score function for mixture of Gaussians"""
    pdf1 = gaussian_pdf(x, mu1, sigma1)
    pdf2 = gaussian_pdf(x, mu2, sigma2)
    score1 = gaussian_score(x, mu1, sigma1)
    score2 = gaussian_score(x, mu2, sigma2)
    
    # Weighted score based on posterior probabilities
    total_pdf = mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1)
    # Avoid division by zero
    total_pdf = np.maximum(total_pdf, 1e-10)
    
    posterior1 = (w1 * pdf1) / total_pdf
    posterior2 = ((1-w1) * pdf2) / total_pdf
    
    return posterior1 * score1 + posterior2 * score2

def smooth_transition(t, pause_fraction=0.1):
    """
    Create a smooth forward-reverse transition with optional pause at extremes
    t should be in [0, 1] for full cycle
    """
    # Add small pauses at the extremes for better visualization
    if t <= pause_fraction:
        return 0.0  # Pause at start
    elif t <= 0.5 - pause_fraction/2:
        # Forward transition
        progress = (t - pause_fraction) / (0.5 - 1.5*pause_fraction)
        return progress
    elif t <= 0.5 + pause_fraction/2:
        return 1.0  # Pause at middle
    elif t <= 1.0 - pause_fraction:
        # Reverse transition
        progress = (t - 0.5 - pause_fraction/2) / (0.5 - 1.5*pause_fraction)
        return 1.0 - progress
    else:
        return 0.0  # Pause at end

def create_forward_reverse_gif(filename='distribution_forward_reverse.gif', 
                              duration=6.0, fps=20, pause_fraction=0.05):
    """
    Create GIF showing full cycle: two-mode → single-mode → two-mode
    with corresponding score function
    """
    
    # Setup
    x = np.linspace(-10, 10, 400)
    total_frames = int(duration * fps)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    def update_frame(frame):
        # Clear both axes
        ax1.clear()
        ax2.clear()
        
        # Cycle parameter (0 to 1 for full forward-reverse cycle)
        cycle_t = frame / (total_frames - 1)
        
        # Get smooth transition parameter
        t = smooth_transition(cycle_t, pause_fraction)
        
        # Determine phase for labeling
        if cycle_t <= 0.5:
            phase = "Forward"
            arrow_direction = "→"
        else:
            phase = "Reverse"
            arrow_direction = "←"
        
        # Evolution parameters
        # t=0: Two separate modes at -4 and +4, small variance
        # t=1: Single mode at 0, large variance
        mu1 = -4 * (1 - t)  # Move from -4 to 0 and back
        mu2 = 4 * (1 - t)   # Move from +4 to 0 and back
        sigma1 = 0.8 + 1.5 * t  # Variance from 0.8 to 2.3 and back
        sigma2 = 0.8 + 1.5 * t  # Variance from 0.8 to 2.3 and back
        
        # Weight evolution (keep equal weights)
        w1 = 0.5
        
        # Calculate current distribution and score
        if abs(t - 1.0) > 0.01:  # Use mixture except when very close to single mode
            current_pdf = mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1)
            current_score = mixture_score(x, mu1, sigma1, mu2, sigma2, w1)
        else:  # Pure single Gaussian when modes overlap
            current_pdf = gaussian_pdf(x, 0, 2.3)
            current_score = gaussian_score(x, 0, 2.3)
        
        # Plot 1: Distribution
        ax1.plot(x, current_pdf, 'b-', linewidth=3, label='p(x)')
        ax1.fill_between(x, current_pdf, alpha=0.3, color='blue')
        
        ax1.set_title(f'Distribution Evolution ({phase} {arrow_direction})')
        ax1.set_xlabel('x')
        ax1.set_ylabel('p(x)')
        ax1.set_ylim(0, 0.3)
        ax1.set_xlim(-8., 8.)
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Score Function
        ax2.plot(x, current_score, 'r-', linewidth=3, label='∇log p(x)')
        ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        
        # Add arrows to show direction of probability flow
        x_arrows = x[::35]  # Subsample for arrows
        score_arrows = current_score[::35]
        
        # Scale arrows for visibility
        arrow_scale = 0.8
        for i, (xi, si) in enumerate(zip(x_arrows, score_arrows)):
            if abs(si) > 0.03:  # Only show significant scores
                ax2.arrow(xi, -0.05, arrow_scale * si, 0, 
                         head_width=0.03, head_length=0.15, 
                         fc='red', ec='red', alpha=0.7)
        
        ax2.set_title(f'Score Function: ∇log p(x) ({phase} {arrow_direction})')
        ax2.set_xlabel('x')
        ax2.set_ylabel('∇log p(x)')
        ax2.set_ylim(-8., 8.)
        ax2.set_xlim(-8., 8.)
        ax2.grid(True, alpha=0.3)
        
        
        plt.tight_layout()
    
    # Create animation
    print(f"Creating forward-reverse animation with {total_frames} frames...")
    print(f"Duration: {duration}s, FPS: {fps}")
    print(f"Pause fraction at extremes: {pause_fraction:.1%}")
    
    anim = FuncAnimation(fig, update_frame, frames=total_frames, 
                        interval=1000/fps, blit=False)
    
    # Save as GIF
    print(f"Saving GIF to {filename}...")
    writer = PillowWriter(fps=fps)
    anim.save(filename, writer=writer)
    
    plt.close()
    print(f"Forward-reverse GIF saved successfully!")
    
    return filename

    
# Create the forward-reverse GIF
gif_filename = create_forward_reverse_gif(
    filename='gaussian_forward_reverse_cycle.gif',
    duration=8.0,     # 8 seconds total
    fps=25,           # 15 frames per second
    pause_fraction=0.08  # 8% pause at each extreme
)

Creating forward-reverse animation with 120 frames...
Duration: 6.0s, FPS: 20
Pause fraction at extremes: 8.0%
Saving GIF to images/gaussian_forward_reverse_cycle.gif...


/tmp/ipykernel_1537642/3333514312.py:143: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Forward-reverse GIF saved successfully!


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import os

def gaussian_pdf(x, mu, sigma):
    """Calculate Gaussian probability density function"""
    return np.exp(-0.5 * ((x - mu) / sigma)**2) / (sigma * np.sqrt(2 * np.pi))

def gaussian_score(x, mu, sigma):
    """Calculate score function (gradient of log pdf) for Gaussian: ∇log p(x) = -(x-μ)/σ²"""
    return -(x - mu) / (sigma**2)

def mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1=0.5):
    """Calculate mixture of two Gaussians"""
    return w1 * gaussian_pdf(x, mu1, sigma1) + (1-w1) * gaussian_pdf(x, mu2, sigma2)

def mixture_score(x, mu1, sigma1, mu2, sigma2, w1=0.5):
    """Calculate score function for mixture of Gaussians"""
    pdf1 = gaussian_pdf(x, mu1, sigma1)
    pdf2 = gaussian_pdf(x, mu2, sigma2)
    score1 = gaussian_score(x, mu1, sigma1)
    score2 = gaussian_score(x, mu2, sigma2)
    
    # Weighted score based on posterior probabilities
    total_pdf = mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1)
    # Avoid division by zero
    total_pdf = np.maximum(total_pdf, 1e-10)
    
    posterior1 = (w1 * pdf1) / total_pdf
    posterior2 = ((1-w1) * pdf2) / total_pdf
    
    return posterior1 * score1 + posterior2 * score2

def create_reverse_only_gif(filename='reverse_process.gif', 
                           duration=4.0, fps=20, pause_fraction=0.1):
    """
    Create GIF showing ONLY reverse process: single-mode → two-mode
    """
    
    # Setup
    x = np.linspace(-10, 10, 400)
    total_frames = int(duration * fps)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    def update_frame(frame):
        # Clear both axes
        ax1.clear()
        ax2.clear()
        
        # Time parameter (0 to 1 for reverse process)
        time_t = frame / (total_frames - 1)
        
        # Add pause at start and end
        if time_t <= pause_fraction:
            t = 1.0  # Start at single mode
        elif time_t >= 1.0 - pause_fraction:
            t = 0.0  # End at two modes
        else:
            # Transition from 1.0 to 0.0 (reverse process)
            progress = (time_t - pause_fraction) / (1.0 - 2*pause_fraction)
            t = 1.0 - progress
        
        # Evolution parameters
        # t=1: Single mode at 0, large variance (START)
        # t=0: Two separate modes at -4 and +4, small variance (END)
        mu1 = -4 * (1 - t)  # Move from 0 to -4
        mu2 = 4 * (1 - t)   # Move from 0 to +4
        sigma1 = 0.8 + 1.5 * t  # Variance from 2.3 to 0.8
        sigma2 = 0.8 + 1.5 * t  # Variance from 2.3 to 0.8
        
        # Weight evolution (keep equal weights)
        w1 = 0.5
        
        # Calculate current distribution and score
        if abs(t - 1.0) > 0.01:  # Use mixture except when very close to single mode
            current_pdf = mixture_pdf(x, mu1, sigma1, mu2, sigma2, w1)
            current_score = mixture_score(x, mu1, sigma1, mu2, sigma2, w1)
        else:  # Pure single Gaussian when modes overlap
            current_pdf = gaussian_pdf(x, 0, 2.3)
            current_score = gaussian_score(x, 0, 2.3)
        
        # Plot 1: Distribution
        ax1.plot(x, current_pdf, 'b-', linewidth=3, label='p(x)')
        ax1.fill_between(x, current_pdf, alpha=0.3, color='blue')
        
        ax1.set_title('Distribution Evolution (Reverse)')
        ax1.set_xlabel('x')
        ax1.set_ylabel('p(x)')
        ax1.set_ylim(0, 0.3)
        ax1.set_xlim(-8., 8.)
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Score Function
        ax2.plot(x, current_score, 'r-', linewidth=3, label='∇log p(x)')
        ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        
        ax2.set_title('Score Function: ∇log p(x) (Reverse)')
        ax2.set_xlabel('x')
        ax2.set_ylabel('∇log p(x)')
        ax2.set_ylim(-8., 8.)
        ax2.set_xlim(-8., 8.)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
    
    # Create animation
    print(f"Creating reverse-only animation with {total_frames} frames...")
    print(f"Duration: {duration}s, FPS: {fps}")
    print(f"Pause fraction at extremes: {pause_fraction:.1%}")
    
    anim = FuncAnimation(fig, update_frame, frames=total_frames, 
                        interval=1000/fps, blit=False)
    
    # Save as GIF
    print(f"Saving GIF to {filename}...")
    writer = PillowWriter(fps=fps)
    anim.save(filename, writer=writer)
    
    plt.close()
    print(f"Reverse-only GIF saved successfully!")
    
    return filename

# Create the reverse-only GIF
gif_filename = create_reverse_only_gif(
    filename='gaussian_reverse_only.gif',
    duration=8.0,     # 4 seconds total
    fps=25,           # 15 frames per second
    pause_fraction=0.1  # 10% pause at each extreme
)

Creating reverse-only animation with 200 frames...
Duration: 8.0s, FPS: 25
Pause fraction at extremes: 10.0%
Saving GIF to images/gaussian_reverse_only.gif...


/tmp/ipykernel_1537642/1164704065.py:107: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Reverse-only GIF saved successfully!
